In [1]:
# this code implements a deep neural network for feature learning
# this code comes from Vishwa's class on constructing a data loader and creating a model

# Some common system imports
import os
import sys
import importlib
import time
import csv

# Numeric computing
import numpy as np

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from scipy.io import wavfile

# pytorch
import torch
import torch.utils.data as tdata
from torchaudio import transforms

# Plotting (if we want it)
import matplotlib.pyplot as plt

# importing our own modules
import audio_datasets as ads

# for one-hot vectore
import torch.nn.functional as func

from sklearn.datasets import load_digits
from sklearn.decomposition import KernelPCA

In [2]:
# loading the raw data for feature learning
training_path = os.path.join(os.getcwd(), "..", "training_data/data") #need to change this back before pushing ***
files = os.listdir(training_path)
wav_files = []
file_type = "wav"
LABELS = {"neutral": 0, "calm": 1, "happy": 2, "sad": 3, "angry": 4, "fearful": 5, "disgust": 6, "surprised": 7}

for file in files:
    curr_path = os.path.join(training_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        for label in LABELS.keys():
            if label in file:
                wav_files.append((file, LABELS[label]))

data_array = []
label_array = []
for data in wav_files: #in the my_datasets code
    data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])
    label_array.append(data[1])

max_len = max([len(data) for data in data_array])
data_array = [np.resize(data, 48000*5) for data in data_array]
print(len(data_array[0]))

/var/folders/q8/zv6syryd4jb5q3z96sz1_lm80000gn/T/ipykernel_16486/3951241112.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])


240000


In [17]:
#setting up the CNN
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        #convolutional layers
        KERNEL = 5
        self.conv1 = torch.nn.Conv2d(1, 8, KERNEL)
        #self.conv2 = torch.nn.Conv2d(1, 1, KERNEL)
        #self.conv3 = torch.nn.Conv2d(1, 1, KERNEL)

        #max pooling layer
        self.pooling_layer = torch.nn.MaxPool2d(3)
        
        #fully connected layers
        self.fc1 = torch.nn.Linear(in_features=38304, out_features=500)
        self.fc2 = torch.nn.Linear(in_features=500, out_features=50)
        self.fc3 = torch.nn.Linear(in_features=50, out_features=8)

    def forward(self, x):
        
        conv_x1 = torch.relu(self.conv1(x))
        #conv_x2 = torch.relu(self.conv2(conv_x1))
        #conv_x3 = torch.relu(self.conv1(conv_x2))

        pooled_x = self.pooling_layer(conv_x1)

        fc_layer_in = pooled_x.reshape((pooled_x.shape[0], -1))
        
        fc_x1 = torch.relu(self.fc1(fc_layer_in))
        fc_x2 = torch.relu(self.fc2(fc_x1))
        fc_x3 = self.fc3(fc_x2)
        
        return torch.softmax(fc_x3, dim=0)
    

In [8]:
# split into training and testing
#transformer = KernelPCA(n_components=40, kernel='cosine')
N_MELS = 40
mfcs = transforms.MelSpectrogram(sample_rate=48000, n_fft=200, n_mels = N_MELS)
mfcc = transforms.MFCC(sample_rate=48000, n_mfcc=40)
mfcc_data = []
for data in data_array:
    mfcc_image = mfcc(torch.tensor(data).float())
    mfcc_data.append(np.array(mfcc_image))
#mfcc_data_array = [np.array(mfcs(torch.tensor(data).float())) for data in data_array] #transform into mfcc before model
train_data, test_data, train_labels, test_labels = train_test_split(mfcc_data, label_array, train_size=0.75, test_size=0.25)
#train_data = mfcc_data_array
#train_labels = label_array
#print(data.shape, train_data.shape, test_data.shape)
for i in range(len(train_data)):
    print(train_data[i].shape)
    train_data[i] = train_data[i].reshape(1, N_MELS, 1201)
for i in range(len(test_data)):
    test_data[i] = test_data[i].reshape(1, N_MELS, 1201)
train_ten, test_ten = torch.tensor(train_data), torch.tensor(test_data)
train_y_ten, test_y_ten = torch.tensor(train_labels), torch.tensor(test_labels)
print(train_ten.shape)
train_dataset = tdata.TensorDataset(train_ten, train_y_ten)

train_loader = tdata.DataLoader(train_dataset, batch_size=50) #this is the data loader (just doing 1 point at a time)

/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (40) may be set too high. Or, the value for `n_freqs` (101) may be set too low.
  warnings.warn(
/Users/davisjackson/Desktop/ELEC301_Project/ELEC301_Project_Env/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)
(40, 1201)

In [15]:
print(train_ten.shape)
print(test_ten.shape)
print(train_y_ten.shape)
print(test_y_ten.shape)

torch.Size([843, 1, 40, 1201])
torch.Size([282, 1, 40, 1201])
torch.Size([843])
torch.Size([282])


In [21]:
n_epochs = 100 #essentially number of iterations

#instantiating a model
model = CNN()

#loss function
criterion = torch.nn.CrossEntropyLoss() #binary cross-entropy loss, clamps log values to protect against extremes

#optimizer mechanism
optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())

loss_array = np.zeros(n_epochs)

for epoch_idx in range(n_epochs):
  print(epoch_idx)
  for batch in train_loader:
    data_batch, labels_batch = batch

    #predict
    labels_pred = model(data_batch.to(torch.float32))
   
    #compute loss
    loss = criterion(labels_pred, labels_batch) #note: cross entropy is not symmetric

    loss_array[epoch_idx] += loss.item()/data_batch.shape[0] #mean of loss
 
    #backprop
    optimizer.zero_grad()
    loss.backward() #backprop from pytorch
    optimizer.step() #all params optimized
  #if epoch_idx%50 == 0:
  #  fig.data[0].y = loss_array[:epoch_idx]
  #  fig.update_yaxes(type='log')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [22]:
test_pred = []
test_dataset = tdata.TensorDataset(test_ten, test_y_ten)
test_loader = tdata.DataLoader(test_dataset, batch_size=50) 
with torch.no_grad():
  for test in test_loader:
    # convolutional layer
    
    label_histogram = model(test[0].to(torch.float32))
    for histogram in label_histogram:
      test_pred.append(np.argmax(histogram).item())


test_pred = torch.tensor(test_pred)
print(classification_report(test_y_ten, test_pred))

              precision    recall  f1-score   support

           0       0.18      0.25      0.21        20
           1       0.55      0.15      0.23        41
           2       0.20      0.54      0.29        35
           3       0.18      0.33      0.24        39
           4       0.89      0.24      0.37        34
           5       0.30      0.08      0.13        36
           6       0.58      0.38      0.45        40
           7       0.44      0.38      0.41        37

    accuracy                           0.29       282
   macro avg       0.41      0.29      0.29       282
weighted avg       0.43      0.29      0.30       282

